In [1]:
# pip install plotly

In [2]:
#pip install statsmodels

In [3]:
#pip install streamlit

In [4]:
#pip install yfinance

In [5]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go
from statsmodels.tsa.statespace.varmax import VARMAX
import pickle
import streamlit as st
import warnings
warnings.filterwarnings('ignore')

In [6]:
pd.set_option('display.max_rows', None)

In [7]:
# Fetch data
def fetch_stock_data(ticker, start_date='2023-12-02', end_date='2023-12-10'):
    stock = yf.Ticker(ticker)
    df = stock.history(start=start_date, end=end_date, interval='1h')
    return df

# Calculate VWAP
def calculate_vwap(df):
    df['VWAP'] = (df['Close'] * df['Volume']).cumsum() / df['Volume'].cumsum()
    return df

# Prepare data for VARMAX
def prepare_data(stock_data):
    combined_data = pd.DataFrame()
    for ticker in stock_data:
        combined_data[f'{ticker}_close'] = stock_data[ticker]['Close']
        combined_data[f'{ticker}_vwap'] = stock_data[ticker]['VWAP']
    return combined_data

In [8]:
# Main function
def main():
    st.title('Stock Price Analysis and Forecasting')
    
    # Fetch and process data
    tickers = ['APX.AX', 'TAH.AX']
    stock_data = {}
    
    for ticker in tickers:
        df = fetch_stock_data(ticker)
        df = calculate_vwap(df)
        stock_data[ticker] = df
    
    # Prepare data for modeling
    model_data = prepare_data(stock_data)
    
    # Train VARMAX model
    model = VARMAX(model_data, order=(1, 1))
    model_fit = model.fit(disp=False)
    
    # Save model
    with open('varmax_model.pkl', 'wb') as f:
        pickle.dump(model_fit, f)
    
    # Predict next day
    steps = 24  # 24 hours
    forecast = model_fit.forecast(steps=steps)
    
    # Create plots
    selected_ticker = st.selectbox('Select Stock:', tickers)
    
    fig = go.Figure()
    
    # Plot historical data
    fig.add_trace(go.Scatter(
        x=stock_data[selected_ticker].index,
        y=stock_data[selected_ticker]['Close'],
        name='Closing Price',
        line=dict(color='blue')
    ))
    
    fig.add_trace(go.Scatter(
        x=stock_data[selected_ticker].index,
        y=stock_data[selected_ticker]['VWAP'],
        name='VWAP',
        line=dict(color='red')
    ))
    
    # Plot predictions
    future_dates = pd.date_range(
        start='2024-12-11 00:00:00',
        end='2024-12-11 23:00:00',
        freq='H'
    )
    
    fig.add_trace(go.Scatter(
        x=future_dates,
        y=forecast[f'{selected_ticker}_close'],
        name='Predicted Price',
        line=dict(color='green', dash='dash')
    ))
    
    fig.update_layout(
        title=f'{selected_ticker} Stock Analysis',
        xaxis_title='Date',
        yaxis_title='Price',
        template='plotly_white'
    )
    
    st.plotly_chart(fig)
    
    # Display metrics
    st.subheader('Forecast Metrics')
    st.write('Last Closing Price:', stock_data[selected_ticker]['Close'].iloc[-1])
    st.write('Predicted Closing Price (End of Next Day):', 
             forecast[f'{selected_ticker}_close'].iloc[-1])

In [9]:
if __name__ == '__main__':
    main()

2024-12-11 13:56:49.279 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-11 13:56:49.581 
  command:

    streamlit run /Users/tongaimutengwa/Library/Python/3.12/lib/python/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-12-11 13:56:49.582 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-11 13:56:51.226 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-11 13:56:51.227 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-11 13:56:51.228 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-11 13:56:51.228 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-11 13:56:51.229